# Predicting Pitches
For our project, we'll aim to solve the problem of predicting the next pitch a Major League Baseball Pitcher will throw during an at bat

In [1]:
import utils.utils as utils
import pandas as pd

## Part 1: Preprocessing the data

In [2]:
# get the data
pitch_data = utils.get_pitch_data()
pitch_data.head()

,pitch_data_id,season,pitcher_id,team_id,team_abbrev,era,wins,losses,throws,b1_id,...,is_runner_on_second,is_runner_on_third,runs_pitcher_team,runs_batter_team,game_id,inning_id,half_inning_id,at_bat_id,gid,p1_pitch_type
0,109192,2018,502706,137,SF,4.6,0.0,2.0,L,527043,...,False,False,0,0,e6bf7ee8-8103-45b1-82e6-f91f97152565,32c550f7-8f86-4374-a716-f82bd1077912,5d6be1db-7ba5-49d0-bc6b-760595b01f95,d800c0f1-85e6-4a0c-a37f-1661d10328d5,gid_2018_04_21_sfnmlb_anamlb_1,SI
1,109193,2018,502706,137,SF,4.6,0.0,2.0,L,527043,...,False,False,0,0,e6bf7ee8-8103-45b1-82e6-f91f97152565,32c550f7-8f86-4374-a716-f82bd1077912,5d6be1db-7ba5-49d0-bc6b-760595b01f95,d800c0f1-85e6-4a0c-a37f-1661d10328d5,gid_2018_04_21_sfnmlb_anamlb_1,SI
2,109194,2018,502706,137,SF,4.6,0.0,2.0,L,527043,...,False,False,0,0,e6bf7ee8-8103-45b1-82e6-f91f97152565,32c550f7-8f86-4374-a716-f82bd1077912,5d6be1db-7ba5-49d0-bc6b-760595b01f95,d800c0f1-85e6-4a0c-a37f-1661d10328d5,gid_2018_04_21_sfnmlb_anamlb_1,KC
3,109195,2018,502706,137,SF,4.6,0.0,2.0,L,527043,...,False,False,0,0,e6bf7ee8-8103-45b1-82e6-f91f97152565,32c550f7-8f86-4374-a716-f82bd1077912,5d6be1db-7ba5-49d0-bc6b-760595b01f95,d800c0f1-85e6-4a0c-a37f-1661d10328d5,gid_2018_04_21_sfnmlb_anamlb_1,SI
4,109196,2018,502706,137,SF,4.6,0.0,2.0,L,527043,...,False,False,0,0,e6bf7ee8-8103-45b1-82e6-f91f97152565,32c550f7-8f86-4374-a716-f82bd1077912,5d6be1db-7ba5-49d0-bc6b-760595b01f95,d800c0f1-85e6-4a0c-a37f-1661d10328d5,gid_2018_04_21_sfnmlb_anamlb_1,SI


In [3]:
pitch_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 72 columns):
pitch_data_id          10000 non-null int64
season                 10000 non-null int64
pitcher_id             10000 non-null int64
team_id                10000 non-null object
team_abbrev            10000 non-null object
era                    10000 non-null float64
wins                   9989 non-null float64
losses                 9989 non-null float64
throws                 10000 non-null object
b1_id                  10000 non-null int64
b1_team_id             10000 non-null object
b1_stand               10000 non-null object
b1_height              10000 non-null int64
b1_bats                10000 non-null object
b1_avg                 10000 non-null float64
b1_hr                  10000 non-null int64
b1_rbi                 10000 non-null int64
b1_bat_order           9490 non-null float64
b1_game_position       9490 non-null object
p1_pitch_id            10000 non-null objec

In [4]:
cols_to_drop=['p1_pitch_id','p0_pitch_id','result_type','pitch_data_id','season','team_id','game_id','inning_id','half_inning_id','at_bat_id','gid']
# drop unneeded cols
pitch_data = utils.drop_columns_by_list(pitch_data,cols_to_drop)
# drop pickoff attempts
pitch_data = utils.drop_pickoffs(pitch_data)
# set category of cols
pitch_data = utils.set_dtypes(pitch_data)
# drop nans
pitch_data = pitch_data.dropna()


In [7]:
len(pitch_data.pitcher_id.unique())

451

In [5]:
pitch_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8830 entries, 0 to 9999
Data columns (total 61 columns):
pitcher_id             8830 non-null object
team_abbrev            8830 non-null object
era                    8830 non-null float64
wins                   8830 non-null float64
losses                 8830 non-null float64
throws                 8830 non-null object
b1_id                  8830 non-null object
b1_team_id             8830 non-null object
b1_stand               8830 non-null int64
b1_height              8830 non-null int64
b1_bats                8830 non-null object
b1_avg                 8830 non-null float64
b1_hr                  8830 non-null int64
b1_rbi                 8830 non-null int64
b1_bat_order           8830 non-null float64
b1_game_position       8830 non-null object
p1_pitch_seqno         8830 non-null int64
p0_pitch_seqno         8830 non-null float64
p0_inning              8830 non-null float64
result_type_simple     8830 non-null object
x          

In [6]:
pitch_types = pitch_data.loc[:,'p1_pitch_type']
Y = utils.encode_pitch_types(pitch_types)
X = pitch_data.drop('p1_pitch_type',axis=1)
# one hot encode necessary cols
X = utils.one_hot_encode(X)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8830 entries, 0 to 9999
Columns: 1053 entries, era to inning_14
dtypes: float64(35), int64(15), uint8(1003)
memory usage: 11.9 MB


In [23]:
X_train, X_test, y_train, y_test = utils.split_dataset_into_train_and_test(X,Y)

In [24]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7049 entries, 3251 to 8313
Columns: 18700 entries, era to inning_15
dtypes: float64(35), int64(15), uint8(18650)
memory usage: 128.1 MB


## Part2: Train an XGBoost multiclassifier

In [25]:
import models.xgboost_model as xgb_model

In [26]:
xgb_multi_class_model = xgb_model.get_multi_class_classifier_model()

In [ ]:
xgb_model.fit_multi_class_model(model=xgb_multi_class_model,x_train=X_train,y_train=y_train,x_test=X_test,y_test=y_test,save_location='./training/xgb_multi_class.bin')